In [1]:
!pip install --quiet PyPDF2
print('pip install PDF 2 complete.')

pip install PDF 2 complete.


In [2]:
# https://stackoverflow.com/a/59348028
import re
import time

import requests
from arrow import now
from bs4 import BeautifulSoup

URL = 'https://www.archives.gov/research/pentagon-papers'

def remove_duplicates(l): # remove duplicates and unURL string
    for item in l:
        match = re.search("(?P<url>https?://[^\s]+)", item)
        if match is not None:
            links.append((match.group("url")))

SKIP = False

time_start = now()
if not SKIP:
    source_code = requests.get(url=URL)
    soup = BeautifulSoup(source_code.content, 'lxml')
    data = []
    links = []


    for link in soup.find_all('a', href=True):
        data.append(str(link.get('href')))
    flag = True
    remove_duplicates(data)
    while flag:
        try:
            for link in links:
                for j in soup.find_all('a', href=True):
                    temp = []
                    source_code = requests.get(link)
                    soup = BeautifulSoup(source_code.content, 'lxml')
                    temp.append(str(j.get('href')))
                    remove_duplicates(temp)

                    if len(links) > 162: # set limitation to number of URLs
                        break
                if len(links) > 162:
                    break
            if len(links) > 162:
                break
        except Exception as e:
            print(e)
            if len(links) > 162:
                break

    FILES = [item for item in links if item.endswith('.pdf')]

print('got {} file links in {}'.format(len(FILES), now() - time_start))

got 71 file links in 0:00:07.402127


In [3]:
from urllib.request import urlretrieve
from os.path import basename
from os.path import exists
from arrow import now

SKIP = False
TARGET = '/kaggle/working/'

time_start = now()
for url in FILES:
    short_name = basename(url)
    target = TARGET + short_name
    if not exists(path=target):
        print('downloading {} to {}'.format(url, target))
        urlretrieve(url, target)
print('done in {}'.format(now() - time_start))

downloading https://nara-media-001.s3.amazonaws.com/arcmedia/research/pentagon-papers/Pentagon-Papers-Index.pdf to /kaggle/working/Pentagon-Papers-Index.pdf
downloading https://nara-media-001.s3.amazonaws.com/arcmedia/research/pentagon-papers/Pentagon-Papers-Part-I.pdf to /kaggle/working/Pentagon-Papers-Part-I.pdf
downloading https://nara-media-001.s3.amazonaws.com/arcmedia/research/pentagon-papers/Pentagon-Papers-Part-II.pdf to /kaggle/working/Pentagon-Papers-Part-II.pdf
downloading https://nara-media-001.s3.amazonaws.com/arcmedia/research/pentagon-papers/Pentagon-Papers-Part-III.pdf to /kaggle/working/Pentagon-Papers-Part-III.pdf
downloading https://nara-media-001.s3.amazonaws.com/arcmedia/research/pentagon-papers/Pentagon-Papers-Part-IV-A-1.pdf to /kaggle/working/Pentagon-Papers-Part-IV-A-1.pdf
downloading https://nara-media-001.s3.amazonaws.com/arcmedia/research/pentagon-papers/Pentagon-Papers-Part-IV-A-2.pdf to /kaggle/working/Pentagon-Papers-Part-IV-A-2.pdf
downloading https://na

In [4]:
import PyPDF2
import pandas as pd
from glob import glob
from os.path import basename
from arrow import now

GLOB = '/kaggle/working/*.pdf'
time_start = now()

result = []
for input_file in glob(pathname=GLOB):
    short_name = basename(input_file)
    print('{} : processing : {}'.format(now().time(), input_file))

    with open(file=input_file, mode='rb') as file_object:
        reader = PyPDF2.PdfReader(file_object)

        for number, page in enumerate(reader.pages):
            series = pd.Series(data=[short_name, number, page.extract_text()], index=['file', 'page', 'text'])
            result.append(series)


df = pd.DataFrame(data=result)
print('done in {}'.format(now() - time_start))

df.shape

19:26:02.081236 : processing : /kaggle/working/Pentagon-Papers-Part-V-A-Vol-IID.pdf
19:26:48.579795 : processing : /kaggle/working/Pentagon-Papers-Part-I.pdf
19:27:59.226075 : processing : /kaggle/working/Pentagon-Papers-Part-IV-C-10.pdf
19:28:10.443985 : processing : /kaggle/working/Pentagon-Papers-Part-IV-B-3.pdf
19:29:04.815656 : processing : /kaggle/working/Pentagon-Papers-Part-V-B-2b.pdf
19:30:45.750497 : processing : /kaggle/working/Pentagon-Papers-Part-V-B-3b.pdf
19:33:36.964241 : processing : /kaggle/working/Pentagon-Papers-Part-IV-C-6-a.pdf
19:34:30.343489 : processing : /kaggle/working/Pentagon-Papers-Part-II.pdf
19:34:58.042635 : processing : /kaggle/working/Pentagon-Papers-Part-IV-C-2a.pdf
19:35:21.900665 : processing : /kaggle/working/Pentagon-Papers-Part-V-A-Vol-IB.pdf
19:35:40.700101 : processing : /kaggle/working/Pentagon-Papers-Part-VI-A.pdf
19:36:06.071240 : processing : /kaggle/working/Pentagon-Papers-Part-V-A-Vol-IC.pdf
19:36:23.680070 : processing : /kaggle/working

(8292, 3)

In [5]:
df = df.sort_values(ascending=True, by=['file', 'page']).reset_index(drop=True)
df.head()

,file,page,text
0,Pentagon-Papers-Index.pdf,0,".. Declassified per Executive Order 13526, Sec..."
1,Pentagon-Papers-Index.pdf,1,TLE \nF<HM \nRTC \nJC \n[J \n[J Declassified p...
2,Pentagon-Papers-Index.pdf,2,"Declassified per Executive Order 13526, Sectio..."
3,Pentagon-Papers-Index.pdf,3,"Declassified per Executive Order 13526, Sectio..."
4,Pentagon-Papers-Index.pdf,4,"Declassified per Executive Order 13526, Sectio..."


In [6]:
from nltk.tokenize import sent_tokenize

df['text'] = df['text'].apply(func=sent_tokenize)
df = df.explode(column='text', ignore_index=True).dropna()
df['text'] = df['text'].str.replace('\n', ' ')
df['token count'] = df['text'].str.split().str.len()

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 215535 entries, 0 to 215536
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   file         215535 non-null  object
 1   page         215535 non-null  int64 
 2   text         215535 non-null  object
 3   token count  215535 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 8.2+ MB


In [7]:
from plotly.express import histogram
histogram(data_frame=df, x='token count', log_y=True)

In [8]:
%env TOKENIZERS_PARALLELISM=false
!pip install --quiet keybert
print('pip install keybert complete.')

env: TOKENIZERS_PARALLELISM=false
pip install keybert complete.


In [9]:
from arrow import now
from keybert import KeyBERT
from sklearn.feature_extraction.text import TfidfVectorizer

MAX_DF = 1.0
MIN_DF = 10 # we can tune this somewhat depending on the size of the corpus
MODEL = 'all-MiniLM-L12-v2'
STOP_WORDS = 'english'
# we use the clean text for keywords even though we show a truncated original message
DOCS = df['text'].values.tolist()

model_start = now()
model = KeyBERT(model=MODEL,)
# we probably don't need to increase the max sequence length from the default of 128
vectorizer = TfidfVectorizer(ngram_range=(1, 1), stop_words=STOP_WORDS, min_df=MIN_DF, max_df=MAX_DF, )
document_embeddings, word_embeddings = model.extract_embeddings(docs=DOCS, vectorizer=vectorizer, )
print('embedding time: {}'.format(now() - model_start))
print('we have {} documents and {} words.'.format(len(document_embeddings), len(word_embeddings)))
keywords = model.extract_keywords(docs=DOCS, top_n=1, stop_words=STOP_WORDS, vectorizer=vectorizer,
                                  doc_embeddings=document_embeddings, word_embeddings=word_embeddings, min_df=MIN_DF, )
print('model time: {}'.format(now() - model_start))
df['keyword'] = [keyword[0][0] if len(keyword) else '-none-' for keyword in keywords]


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

embedding time: 1:33:38.451584
we have 215535 documents and 13650 words.
model time: 1:38:02.697376


In [10]:
df[df['keyword'] != '-none-']['keyword'].value_counts(normalize=True).head(n=10)

keyword
declassified    0.047419
vietnam         0.043911
hanoi           0.018206
viet            0.017349
nwd             0.017146
secret          0.015508
indochina       0.014579
saigon          0.013692
vietnamese      0.010071
laos            0.006515
Name: proportion, dtype: float64